## Data inspection

In [67]:
import pandas as pd
import numpy as np

In [50]:
year_file = pd.ExcelFile("data/R99.xlsx")
year_file.sheet_names

In [80]:
sheets_desc = {"Data": "مشخصات پرسشنامه",
               "P1":"قسمت یکم: خصوصیات اجتماعی اعضای خانوار",
               "P2": "قسمت دوم: مشخصات محل سکونت",
               "P3S01": "قسمت سوم: بخش ۱ هزینه‌های خوراکی و دخانی",
               "P3S02": "قسمت سوم: بخش ۲ هزینه‌های نوشیدنی‌های طبقه‌بندی نشده",
               "P3S03": "قسمت سوم: بخش ۳ هزینه‌های غیرخوراکی",
               "P3S04": "قسمت سوم: بخش ۴ هزینه‌های بخش مسکن",
               "P3S05": "قسمت سوم: بخش ۵ هزینه‌های غیرخوراکی",
               "P3S06": "قسمت سوم: بخش ۶ هزینه‌های غیرخوراکی",
               "P3S07": "قسمت سوم: بخش ۷ هزینه‌های غیرخوراکی",
               "P3S08": "قسمت سوم: بخش ۸ هزینه‌های غیرخوراکی",
               "P3S09": "قسمت سوم: بخش ۹ هزینه‌های غیرخوراکی",
               "P3S10": "قسمت سوم: بخش ۱۰ هزینه‌های غیرخوراکی",
               "P3S11": "قسمت سوم: بخش ۱۱ هزینه‌های غیرخوراکی",
               "P3S12": "قسمت سوم: بخش ۱۲ هزینه‌های غیرخوراکی",
               "P3S13": "قسمت سوم: بخش ۱۳ سایر هزینه‌ها و انتقالات",
               "P3S14": "قسمت سوم: بخش ۱۴ سرمایه‌گذاری خانوار در ۱۲ ماه گذشته",
               "P4S01": "قسمت چهارم: بخش ۱ درآمد پولی اعضای شاغل خانوار از مشاغل مزد و حقوق بگیری",
               "P4S02": "قسمت چهارم: بخش ۲ درآمد پولی اعضای شاغل خانوار از مشاغل آزاد",
               "P4S03": "قسمت چهارم: بخش ۳ درآمدهای متفرقه خانوار",
               "P4S04": "قسمت چهارم: ستون ۹ بخش ۳ شامل وام و یارانه"
               }

In [95]:
i = 0
sheet_name = list(sheets_desc.keys())[i]
print(f"Sheet name: {sheet_name}, Loaded sheet description: {sheets_desc[sheet_name]}")

Sheet name: Data, Loaded sheet description: مشخصات پرسشنامه


In [96]:
current_df = pd.read_excel(year_file, year_file.sheet_names[i])

In [97]:

lookup = 20001383916
# current_df.loc[(current_df.kilogram + current_df.gram.astype(float)/1000) * current_df.price !=current_df.value, :]
# current_df.gram.replace(r'^\s*$', np.nan, regex=True).astype(float).fillna(0)
# print(current_df)
# print(current_df.dtypes)
current_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18251 entries, 0 to 18250
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Address         18251 non-null  int64  
 1   MahMorajeh      18251 non-null  int64  
 2   Fasl            18251 non-null  int64  
 3   weight          18251 non-null  float64
 4   khanevartype    18251 non-null  int64  
 5   Takmil          18251 non-null  int64  
 6   TakmilDescA     0 non-null      float64
 7   TakmilDescB     3690 non-null   float64
 8   TakmilDescC     980 non-null    object 
 9   Jaygozin        3690 non-null   float64
 10  JaygozinDescA   0 non-null      float64
 11  JaygozinDescB   0 non-null      float64
 12  JaygozinDescC   0 non-null      float64
 13  BlkAbdJaygozin  3690 non-null   float64
 14  RadifJaygozin   3690 non-null   float64
 15  IsTel           14682 non-null  float64
 16  FVam            16209 non-null  float64
 17  province        18251 non-null 

In [98]:
current_df.loc[current_df.Address == lookup, :]

,Address,MahMorajeh,Fasl,weight,khanevartype,Takmil,TakmilDescA,TakmilDescB,TakmilDescC,Jaygozin,JaygozinDescA,JaygozinDescB,JaygozinDescC,BlkAbdJaygozin,RadifJaygozin,IsTel,FVam,province,town
0,20001383916,5,2,170.110792,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,Markazi,1


In [99]:
current_df.loc[~current_df.TakmilDescB.isna(), :]

,Address,MahMorajeh,Fasl,weight,khanevartype,Takmil,TakmilDescA,TakmilDescB,TakmilDescC,Jaygozin,JaygozinDescA,JaygozinDescB,JaygozinDescC,BlkAbdJaygozin,RadifJaygozin,IsTel,FVam,province,town
4,20001383926,5,2,170.110792,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,268.0,25.0,1.0,1.0,Markazi,1
22,20001384226,2,1,170.110792,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,11.0,25.0,1.0,2.0,Markazi,1
25,20001384317,6,2,170.110792,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,10.0,13.0,1.0,1.0,Markazi,1
26,20001384323,6,2,170.110792,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,13.0,21.0,1.0,1.0,Markazi,1
27,20001384324,6,2,170.110792,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,13.0,22.0,1.0,1.0,Markazi,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18235,23006709621,5,2,95.778083,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,19.0,2.0,NaN,Alborz,6
18242,23006709723,9,3,79.815070,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,3.0,20.0,2.0,1.0,Alborz,6
18244,23006709729,9,3,79.815070,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,3.0,27.0,2.0,2.0,Alborz,6
18246,23006709817,1,4,79.815070,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,4.0,14.0,2.0,1.0,Alborz,6


Drop rows that:

TakmilDescB or JaygozinDescA or JaygozinDescB or JaygozinDescC is not nan,
Jaygozin ==2,

However, JaygozinDescC does not appear to have a specific definition.

Because all the above mean the end of census.


In [123]:
current_df = current_df.loc[~current_df.TakmilDescB.isna() | 
                            ~current_df.JaygozinDescA.isna() | 
                            ~current_df.JaygozinDescB.isna() |
                            ~current_df.JaygozinDescC.isna() |
                            current_df.Jaygozin==2, :]


3690

In [119]:
current_df.TakmilDescC.value_counts()

88                                                      577
هشتادو هشت                                              170
هشتاد و هشت                                             111
هشتادوهشت                                                29
هشتاد وهشت                                               18
99                                                        8
نود ونه                                                   5
تلفن ندارد                                                4
کد هشتاد و هشت                                            3
کد 88                                                     3
عدم پاسخگويي                                              3
نبود تلفن                                                 3
کد88                                                      3
شيوع کرونا                                                3
نود و نه                                                  2
شماره اشتباه                                              2
هشتاد و هشت                             

In [120]:
(current_df.Jaygozin==2).sum()

0

In [ ]:
drop df columns